# Introduction

Do not spend too much time trying to get very tiny metrics improvement. Once you have a model with a correct predictive power, you should better spend time explaining your data cleaning & preparation pipeline as well as explanations & visualizations of the results.

The goal is to see your fit with our company culture & engineering needs, spending 50h on an over-complicated approach will not give you bonus points compared to a simple, yet effective, to-the-point solution.

## About the data

The dataset you will be working with is called Emo-DB and can be found [here](http://emodb.bilderbar.info/index-1280.html).

It is a database containing samples of emotional speech in German. It contains samples labeled with one of 7 different emotions: Anger, Boredom, Disgust, Fear, Happiness, Sadness and Neutral. 

Please download the full database and refer to the documentation to understand how the samples are labeled (see "Additional information")
   
The goal of this project is to develop a model which is able to **classify samples of emotional speech**. Feel free to use any available library you would need, but beware of re-using someone else's code without mentionning it!

## Deliverable

The end-goal is to deliver us a zip file containing:
* This report filled with your approach, in the form of an **iPython Notebook**.
* A **5-10 slides PDF file**, containing a technical presentation covering the important aspects of your work
* A Dockerfile which defines a container for the project. The container should handle everything (download the data, run the code, etc...). When running the container it should expose the jupyter notebook on one port and expose a Flask API on another one. The Flask app contains two endpoints:
  - One for training the model
  - One for querying the last trained model with an audio file of our choice in the dataset
* A README.md which should contain the commands to build and run the docker container, as well as how to perform the queries to the API. 
* Any necessary .py, .sh or other files needed to run your code.

# Libraries Loading

In [3]:
import glob
from scipy.io import wavfile
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from IPython.display import clear_output
import numpy as np
from flask import render_template
import traceback
import pickle
import pandas as pd
from IPython.core.debugger import set_trace
from speechpy.feature import mfcc

# Data Preparation & Cleaning

In [4]:
DE2EN = {'W':'A', #Wut-Anger
         'L':'B', #Langeweile-Bordom
         'E':'D', #Ekel-Disgust
         'A':'F', #Angst-Fear
         'F':'H', #Freude-Happiness
         'T':'S',
         'N':'N'} #Traueer-Sadness
EN2DE = {value:key for key,value in DE2EN.items()}
EN2NUM = {item[1]:num for item,num in zip(DE2EN.items(),range(len(DE2EN)))}
DE2NUM = {item[0]:num for item,num in zip(DE2EN.items(),range(len(DE2EN)))}

In [5]:
DATA_DIR = "./data"
WAV_DIR = os.path.join(DATA_DIR,"wav")
temp = wavfile.read("./data/wav/03a01Fa.wav")

In [6]:
def parse_filename(filename):
    """
    parses the attributes of a given sample based on its filename
    """
    speaker_id = int(filename[:2])
    text_id = filename[2:5]
    emotion_de = filename[5]
    emotion_en = DE2EN[emotion_de]
    return speaker_id,text_id,emotion_en

In [7]:
speaker_data = [[3  , 'male',  31],
                [8  , 'female',34 ],
                [9  , 'female',21 ],
                [10 , 'male',  32 ],
                [11 , 'male',  26 ],
                [12 , 'male',  30] ,
                [13 , 'female',32], 
                [14 , 'female',35] ,
                [15 , 'male',  25] ,
                [16 , 'female',31]]
speaker_data = pd.DataFrame(speaker_data,columns = ['speaker_id','sex','age'])
speaker_data.set_index('speaker_id',inplace=True)

In [8]:
def load_pd_data(wav_dir=WAV_DIR):
    for root, dirs, files in os.walk(wav_dir, topdown=False):
        paths = [os.path.join(root,file) for file in files]
        data = []
        for file in files:
            audio_data = wavfile.read(os.path.join(root,file))[1]
            speaker_id,text_id,emotion_en = parse_filename(file)
            row = [speaker_id,text_id,emotion_en,audio_data]
            data.append(row)
    res = pd.DataFrame(data,columns=["speaker_id","text_id","emotion","data"])
    return res.join(speaker_data,on="speaker_id")

In [9]:
def load_data(wav_dir=WAV_DIR):
    data = []
    sfs = []
    targets = []
    for root, dirs, files in os.walk(wav_dir, topdown=False):
        for file in files:
            sf,audio_data = wavfile.read(os.path.join(root,file))
            data.append(audio_data)
            sfs.append(sf)
            target = DE2NUM[file[5].capitalize()]
            targets.append(target)
    datapadd = zeropadd(data,mode='mean')
    return np.array(sfs),datapadd,np.array(targets)

In [10]:
def zeropadd(data,mode='max'):
    if mode == 'max':
        new_len = max([x.shape[0] for x in data])
    else:
        new_len = int(np.round(np.mean([x.shape[0] for x in data])))
    def padd(x):
        diff = abs(new_len - x.shape[0])
        shift = diff %2
        diff //=2
        if x.shape[0] < new_len:
            return np.pad(x,(diff,diff+shift),'constant')
        else:
            return x[diff:-(diff+shift)]
    data_padded = np.zeros((len(data),new_len))
    for i,x in enumerate(data):
        data_padded[i] = padd(x)
    return data_padded

In [11]:
def get_mfcc(data,sfs):
    return np.array([mfcc(x,sf,num_cepstral=39) for x,sf in zip(data,sfs)])

# Feature Engineering & Modeling

In [12]:
def dummy_model(filename):
    data = wavfile.read(os.path.join(WAV_DIR,filename))[1]
    return filename[5]

In [33]:
class CNN_classif(nn.Module):
    def __init__(self):
        super(CNN_classif,self).__init__()
        self.convblock1 = nn.Sequential(
                                nn.Conv2d(1,8,kernel_size=13),
                                nn.BatchNorm2d(8),
                                nn.ReLU())
        self.convblock2 = nn.Sequential(
                                nn.Conv2d(8,8,kernel_size=13),
                                nn.BatchNorm2d(8),
                                nn.ReLU(),
                                nn.MaxPool2d(kernel_size=(2,1)))
        self.convblock3 = nn.Sequential(
                                nn.Conv2d(8,8,kernel_size=13),
                                nn.BatchNorm2d(8),
                                nn.ReLU())
        self.convblock4 = nn.Sequential(
                                nn.Conv2d(8,8,kernel_size=2),
                                nn.BatchNorm2d(8),
                                nn.ReLU(),
                                nn.MaxPool2d(kernel_size=(2,1)))
        self.linblock = nn.Sequential(
                                nn.Flatten(),
                                nn.Linear(896,64),
                                nn.ReLU(),
                                nn.Dropout(0.2),
                                nn.Linear(64,7)
        )        
    def forward(self,x):
        #set_trace()
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        x = self.linblock(x)
        return x

In [50]:
def train_model(model, inputs, targets):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr = 1e-4)
    nb_epochs = 250
    batch_size = 20
    for e in range(nb_epochs):
        clear_output(wait=True)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        accuracy = (predicted == targets).sum().item() / inputs.shape[0] * 100
        print("Progression:{} % Accuracy: {}% ".format(e/nb_epochs*100,accuracy))
        for train_batch,target_batch in zip(inputs.split(batch_size),
                                targets.split(batch_size)):
            output_batch = model(train_batch)
            loss = criterion(output_batch,target_batch)
            
            model.zero_grad()
            loss.backward()
            optimizer.step()

In [15]:
sfs,data,targets = load_data()
data_f = get_mfcc(data,sfs)

/home/ymentha/anaconda3/envs/ML/lib/python3.7/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [48]:
model = CNN_classif()

In [51]:
data_f = torch.Tensor(data_f).unsqueeze(1)
targets = torch.Tensor(targets)
train_model(model,data_f,targets.long())

In [59]:
torch.save(model.state_dict(), "./models/15_04_1605")
models2 = CNN_classif()
models2.load_state_dict(torch.load("./models/15_04_1605"))

# Results & Visualizations

In [283]:
import tensorflow as tf

loss  = tf.keras.losses.BinaryCrossentropy()

loss([0., 0., 1., 1.], [1., 1., 1., 0.])

loss2 = nn.NLLLoss()
input2 = torch.Tensor([[1.0,0.0],[1.0,0.0], [0.0,1.0],[0.0,1.0]])
input2.requires_grad_ = True
target2 = torch.Tensor([1,1,1,0]).long()
loss2(input2, target2)

loss3 = nn.BCELoss(reduction='mean')
input3 = torch.Tensor([0., 0., 1., 1.])
target3 = torch.Tensor([1., 1., 1., 0.])
loss3(input3,target3 )